In [58]:
import math

import numpy
import numpy as np
import scipy


def black_scholes(spot_price, strike_price, risk_rate, mature_time, volitility, callput):
    ddivisor =volitility*mature_time * math.sqrt(mature_time)
    d1 = (math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (ddivisor)
    d2 = (d1 - ddivisor)
    call = (spot_price * scipy.stats.norm.cdf(d1)) - strike_price * math.exp(
        -risk_rate * mature_time) * scipy.stats.norm.cdf(d2)
    put = strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(
        -d2) - spot_price * scipy.stats.norm.cdf(-d1)
    if callput == "call":
        return call
    else:
        return put
    # call = (spot_price * scipy.stats.norm.cdf((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)))) - strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)) - (volitility * math.sqrt(mature_time))))

In [59]:
print(black_scholes(100, 120, 0.05, 0.5, 0.2, "call"))
print(black_scholes(100, 120, 0.05, 0.5, 0.2, "put"))
print(black_scholes(100,94,0.05, 0.5, 0.2, "call"))
print(black_scholes(100,94,0.05, 0.5, 0.2, "put"))
# black_scholes(spot_price,strike_price,risk_rate,mature_time,volatility)

0.03311723312069925
17.070306676520616
8.672275184569173
0.35140691523244527


In [63]:
import math
import numpy
import scipy


def binomial(starting_value, strike, rate, volitility, time, step, type, excersise):
    up = math.exp(volitility * math.sqrt(time / step))
    down = 1 / up
    rate_value = math.exp((-rate) * time / step)
    increase_probability = ((math.exp(rate * (time / step)) - down) / (up - down))
    decrease_probability = 1 - increase_probability

    tree = [[0]]
    tree[0][0] = starting_value
    for cur_depth in range(1, step + 1):
        new_values = []
        old = tree[-1]
        for child in range(0, cur_depth + 1):
            if cur_depth == child:
                new = old[-1] * down
            elif child == 0:
                new = old[0] * up
            else:
                new = ((old[child - 1] * up) + (old[child] * down)) / 2
            new_values.append(new)
        tree.append(new_values)
    children = tree[-1]
    values = []
    for i in children:
        if type == "put":
            if strike - i > 0:
                option = strike - i
            else:
                option = 0
        else:
            if i - strike > 0:
                option = i - strike
            else:
                option = 0
        values.append(option)
    k = step-1
    while k >= 0:
        for j in range(k + 1):
            option = (increase_probability * values[j + 1] + decrease_probability * values[j]) * rate_value
            if type == "put":
                if strike - tree[k][j] > 0:
                    option = strike - tree[k][j]
                else:
                    option = 0
            else:
                if tree[k][j] - strike > 0:
                    option = tree[k][j] - strike
                else:
                    option = 0
            values[j] = option
        k = k - 1
    # print(tree)
    print(values)
    return values[0]


# print(binomial(100, 120, 0.05, 0.2, 0.5,183,"put","e"))
#binomial(100,110,0.05,0.2,1,100,'call','euro')
# binomial(20,21,0.12,0.1,0.5,2,"call","euro")

print(binomial(100, 120, 0.05, 0.2, 0.5, 183, "put", "e"))
print(binomial(100, 120, 0.05, 0.2, 0.5, 183, "call", "e"))
print(binomial(100, 120, 0.05, 0.2, 0.5, 183, "put", "a"))
print(binomial(100, 120, 0.05, 0.2, 0.5, 183, "call", "a"))


[20, 21.039971258639994, 22.069127115092016, 23.087580046430332, 24.095441359999953, 25.092821205581487, 26.079828587429503, 27.05657137618566, 28.02315632066791, 28.979689059537193, 29.926274132842764, 30.86301499344738, 31.790014018333764, 32.707372519793466, 33.61519075649929, 34.51356794446268, 35.40260226787707, 36.282390889848514, 37.15302996301466, 38.014614640053395, 38.86723908408209, 39.71099647894873, 40.54597903941611, 41.37227802123998, 42.18998373114245, 42.999185536681665, 43.79997187601889, 44.59243026758388, 45.376647319639915, 46.152708739749215, 46.92069934413999, 47.68070306697604, 48.43280296953, 49.17708124926115, 49.913619248798796, 50.642497464832275, 51.36379555690853, 52.07759235613807, 52.78396587381053, 53.482993309920545, 54.17475106160501, 54.859314731492546, 55.536759135966165, 56.20715831333999, 56.870585531950965, 57.52711329816639, 58.17681336430819, 58.819756736494746, 59.456013682401235, 60.08565373893919, 60.708745719856275, 61.32535772325692, 61.93

TypeError: 'int' object is not subscriptable

In [60]:
import numpy


def monte_carlo(stock_price,strike_price,time, interest_rate, volitility, type,steps,simulations ):
    volitility *= time
    dt = time/steps
    rn = (interest_rate-0.5*volitility**2)*dt
    vol = volitility*numpy.sqrt(dt)
    z = numpy.random.normal(size=(steps,simulations))
    st = stock_price*numpy.exp(numpy.cumsum(rn+vol*z,axis=0))
    if type=="call":
        values=numpy.maximum(st[-1]-strike_price,0)
    else:
        values=numpy.maximum(strike_price-st[-1],0)
    return  numpy.mean(values)*numpy.exp(-interest_rate*time)




print(monte_carlo(100,120, 0.5, 0.05,0.2,"call", 252,100000))
print(monte_carlo(100,120, 0.5, 0.05,0.2,"put", 252,100000))
print(monte_carlo(100,94, 0.5, 0.05,0.2,"call", 252,100000))
print(monte_carlo(100,94, 0.5, 0.05,0.2,"put", 252,100000))

0.03497425901402458
17.08778409256354
8.724575419897395
0.355205052314832
